In [5]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load the CSV file
df = pd.read_csv('carfinalkub.csv')

# Drop rows with missing lat/long values
df.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Create a base map
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=15)

# Create a dictionary to map each category to a specific color
# This ensures that a specific category always gets the same color
categories = sorted(df['detail_category'].unique())
color_map = {category: color for category, color in zip(categories, folium.
                                                         LinearColormap(
                                                             ['blue', 'green', 'red', 'orange', 'purple'],
                                                             vmin=0,
                                                             vmax=len(categories)-1).colors)}

# Create MarkerCluster groups for each category
for category in categories:
    category_df = df[df['detail_category'] == category]

    # Create a feature group for the category with a specific name
    marker_group = folium.FeatureGroup(name=f'{category}').add_to(m)

    # Add MarkerCluster to the group to handle grouping
    marker_cluster = MarkerCluster().add_to(marker_group)

    for index, row in category_df.iterrows():
        # Get the color for the current category
        color = color_map.get(category, 'gray')  # Use gray as a default color

        # Add a CircleMarker to the cluster
        # You can adjust radius, color, and fill here
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,  # Adjust the size of the circle
            color=color, # Border color
            fill=True,
            fill_color=color, # Fill color
            fill_opacity=0.7,
            popup=f"**Detail:** {row['detail_norm']}<br>**Category:** {category}",
        ).add_to(marker_cluster)

# Add Layer Control to enable filtering
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save('accident_map_categorized.html')

In [9]:
# layered_dashboard_map.py
# deps: pandas, folium
# optional deps: folium.plugins (HeatMap, MarkerCluster)

import pandas as pd
import folium
from folium import FeatureGroup
from folium.plugins import MarkerCluster, HeatMap
from pathlib import Path

CSV_PATH = Path("/content/carfinalkub.csv")
OUT_HTML = Path("car_map_dashboard.html")

# --------- Config ----------
MAX_POINTS = 1000        # cap markers for speed
HEATMAP_MIN_POINTS = 10  # require at least this many points for heatmap layer
HEATMAP_DEFAULT_VISIBLE = False  # 👈 heatmap shown by default? (toggle in LayerControl)
CATEGORY_COL = "detail_category"
DATE_COL = "created_date"
TIME_COL = "created_time"  # optional, will be combined if present
LAT_COL = "Latitude"
LON_COL = "Longitude"
# ---------------------------

def load_df(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # coords
    df[LAT_COL] = pd.to_numeric(df.get(LAT_COL), errors="coerce")
    df[LON_COL] = pd.to_numeric(df.get(LON_COL), errors="coerce")
    df = df.dropna(subset=[LAT_COL, LON_COL]).copy()

    # datetime
    if DATE_COL in df.columns:
        if TIME_COL in df.columns and df[TIME_COL].notna().any():
            dt_str = df[DATE_COL].astype(str).str.strip() + " " + df[TIME_COL].astype(str).str.strip()
            df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)
        else:
            df["event_dt"] = pd.to_datetime(df[DATE_COL], errors="coerce", infer_datetime_format=True)
    else:
        df["event_dt"] = pd.NaT

    return df

def make_popup(row: pd.Series) -> str:
    keys = [
        "place_input", CATEGORY_COL, "vehicle_name_1","vehicle_name_2","vehicle_name_3",
        "person_type_name_1","person_type_name_2","person_type_name_3",
        DATE_COL, TIME_COL, "occurred","ended","is_ambulance","detail"
    ]
    parts = []
    for k in keys:
        if k in row and pd.notna(row[k]):
            v = str(row[k])
            if k == "detail": v = v[:300]
            parts.append(f"<b>{k}:</b> {v}")
    return "<br>".join(parts) if parts else "Location"

def color_for_category(cat: str) -> str:
    # simple deterministic palette
    palette = [
        "#1f77b4","#ff7f0e","#2ca02c","#d62728",
        "#9467bd","#8c564b","#e377c2","#7f7f7f",
        "#bcbd22","#17becf"
    ]
    if pd.isna(cat): return "#7f7f7f"
    idx = abs(hash(str(cat))) % len(palette)
    return palette[idx]

def filter_df(df: pd.DataFrame, start=None, end=None, categories=None, ambulance=None) -> pd.DataFrame:
    out = df.copy()
    if start is not None:
        out = out[out["event_dt"].ge(pd.to_datetime(start, errors="coerce"))]
    if end is not None:
        out = out[out["event_dt"].le(pd.to_datetime(end, errors="coerce"))]
    if categories:
        out = out[out[CATEGORY_COL].isin(categories)]
    if ambulance is not None and "is_ambulance" in out.columns:
        out = out[out["is_ambulance"] == ambulance]
    return out

def add_points_layer(m: folium.Map, df: pd.DataFrame, name: str, cluster=True, show=True) -> None:
    fg = FeatureGroup(name=name, show=show)
    if cluster:
        mc = MarkerCluster(disableClusteringAtZoom=7)
    count = 0
    for _, r in df.iterrows():
        if count >= MAX_POINTS: break
        color = color_for_category(r.get(CATEGORY_COL))
        marker = folium.CircleMarker(
            [r[LAT_COL], r[LON_COL]],
            radius=5, color="black", weight=0.4,
            fill=True, fill_color=color, fill_opacity=0.35,
            popup=folium.Popup(make_popup(r), max_width=420),
        )
        if cluster:
            mc.add_child(marker)
        else:
            fg.add_child(marker)
        count += 1
    if cluster:
        fg.add_child(mc)
    m.add_child(fg)

def add_heatmap_layer(m: folium.Map, df: pd.DataFrame, name: str, show: bool = HEATMAP_DEFAULT_VISIBLE) -> None:
    """
    Wrap HeatMap in a FeatureGroup with a 'name' and 'show' flag so it appears in LayerControl.
    """
    pts = df[[LAT_COL, LON_COL]].dropna().values.tolist()
    if len(pts) >= HEATMAP_MIN_POINTS:
        fg = FeatureGroup(name=name, show=show)
        HeatMap(pts, radius=16, blur=20, max_zoom=9).add_to(fg)
        m.add_child(fg)

def build_map(df: pd.DataFrame, out_html: Path) -> None:
    center = [df[LAT_COL].median(), df[LON_COL].median()]
    # Use a clean non-Google basemap
    m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

    # All points (visible by default)
    add_points_layer(m, df, "All points", cluster=True, show=True)

    # Heatmap (toggable via LayerControl; default visibility set by HEATMAP_DEFAULT_VISIBLE)
    add_heatmap_layer(m, df, "Heatmap (All)", show=HEATMAP_DEFAULT_VISIBLE)

    # Category layers (top N) — each is togglable
    if CATEGORY_COL in df.columns:
        top = (
            df[CATEGORY_COL]
            .value_counts(dropna=False)
            .head(5)
            .index.tolist()
        )
        for cat in top:
            sub = df[df[CATEGORY_COL] == cat]
            add_points_layer(m, sub, f"Category: {cat}", cluster=True, show=False)

    # Legend (category color swatches)
    if CATEGORY_COL in df.columns:
        cats = (
            df[CATEGORY_COL]
            .value_counts(dropna=False)
            .head(6)
            .index.tolist()
        )
        items = "".join(
            f'<div style="margin:4px 0;"><span style="display:inline-block;width:12px;height:12px;background:{color_for_category(c)};border:1px solid #333;margin-right:6px;"></span>{c}</div>'
            for c in cats
        )
        legend = f"""
        <div style="position: fixed; bottom: 24px; left: 24px; z-index: 9999;
            background: white; border: 1px solid #999; padding: 10px 12px; font-size: 13px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.15);">
            <b>{CATEGORY_COL} (top)</b>
            {items}
        </div>
        """
        m.get_root().html.add_child(folium.Element(legend))

    # Layer control (appears after we add all layers)
    folium.LayerControl(collapsed=False).add_to(m)

    # Fit bounds
    bounds = df[[LAT_COL, LON_COL]].dropna().values.tolist()
    if bounds:
        m.fit_bounds(bounds)

    m.save(str(out_html))

if __name__ == "__main__":
    df = load_df(CSV_PATH)
    # Example filters (uncomment/edit as needed):
    # df = filter_df(df, start="2025-01-01", end="2025-12-31",
    #                categories=["Accident","Traffic"], ambulance=1)
    build_map(df, OUT_HTML)
    print(f"Saved: {OUT_HTML.resolve()}")


/tmp/ipython-input-1699161339.py:36: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-1699161339.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)


Saved: /content/car_map_dashboard.html


In [11]:
# layered_dashboard_map.py
# deps: pandas, folium
# optional deps: folium.plugins (HeatMap, MarkerCluster)

import pandas as pd
import folium
from folium import FeatureGroup
from folium.plugins import MarkerCluster, HeatMap
from pathlib import Path

CSV_PATH = Path("/content/no2car.csv")
OUT_HTML = Path("car_map_dashboard2.html")

# --------- Config ----------
MAX_POINTS = 1000        # cap markers for speed
HEATMAP_MIN_POINTS = 10  # require at least this many points for heatmap layer
HEATMAP_DEFAULT_VISIBLE = False  # 👈 heatmap shown by default? (toggle in LayerControl)
CATEGORY_COL = "detail_category"
TIME_PERIOD_COL = "Time Period"
DATE_COL = "created_date"
TIME_COL = "created_time"  # optional, will be combined if present
LAT_COL = "Latitude"
LON_COL = "Longitude"
# ---------------------------

def load_df(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # coords
    df[LAT_COL] = pd.to_numeric(df.get(LAT_COL), errors="coerce")
    df[LON_COL] = pd.to_numeric(df.get(LON_COL), errors="coerce")
    df = df.dropna(subset=[LAT_COL, LON_COL]).copy()

    # datetime
    if DATE_COL in df.columns:
        if TIME_COL in df.columns and df[TIME_COL].notna().any():
            dt_str = df[DATE_COL].astype(str).str.strip() + " " + df[TIME_COL].astype(str).str.strip()
            df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)
        else:
            df["event_dt"] = pd.to_datetime(df[DATE_COL], errors="coerce", infer_datetime_format=True)
    else:
        df["event_dt"] = pd.NaT

    return df

def make_popup(row: pd.Series) -> str:
    keys = [
        "place_input", CATEGORY_COL, "vehicle_name_1","vehicle_name_2","vehicle_name_3",
        "person_type_name_1","person_type_name_2","person_type_name_3",
        DATE_COL, TIME_COL, "occurred","ended","is_ambulance","detail"
    ]
    parts = []
    for k in keys:
        if k in row and pd.notna(row[k]):
            v = str(row[k])
            if k == "detail": v = v[:300]
            parts.append(f"<b>{k}:</b> {v}")
    return "<br>".join(parts) if parts else "Location"

def color_for_category(cat: str) -> str:
    # simple deterministic palette
    palette = [
        "#1f77b4","#ff7f0e","#2ca02c","#d62728",
        "#9467bd","#8c564b","#e377c2","#7f7f7f",
        "#bcbd22","#17becf"
    ]
    if pd.isna(cat): return "#7f7f7f"
    idx = abs(hash(str(cat))) % len(palette)
    return palette[idx]

def filter_df(df: pd.DataFrame, start=None, end=None, categories=None, time_periods=None, ambulance=None) -> pd.DataFrame:
    out = df.copy()
    if start is not None:
        out = out[out["event_dt"].ge(pd.to_datetime(start, errors="coerce"))]
    if end is not None:
        out = out[out["event_dt"].le(pd.to_datetime(end, errors="coerce"))]
    if categories:
        out = out[out[CATEGORY_COL].isin(categories)]
    if time_periods:
        out = out[out[TIME_PERIOD_COL].isin(time_periods)]
    if ambulance is not None and "is_ambulance" in out.columns:
        out = out[out["is_ambulance"] == ambulance]
    return out

def add_points_layer(m: folium.Map, df: pd.DataFrame, name: str, cluster=True, show=True, color_col:str=CATEGORY_COL) -> None:
    fg = FeatureGroup(name=name, show=show)
    if cluster:
        mc = MarkerCluster(disableClusteringAtZoom=7)
    count = 0
    for _, r in df.iterrows():
        if count >= MAX_POINTS: break
        color = color_for_category(r.get(color_col))
        marker = folium.CircleMarker(
            [r[LAT_COL], r[LON_COL]],
            radius=5, color="black", weight=0.4,
            fill=True, fill_color=color, fill_opacity=0.35,
            popup=folium.Popup(make_popup(r), max_width=420),
        )
        if cluster:
            mc.add_child(marker)
        else:
            fg.add_child(marker)
        count += 1
    if cluster:
        fg.add_child(mc)
    m.add_child(fg)

def add_heatmap_layer(m: folium.Map, df: pd.DataFrame, name: str, show: bool = HEATMAP_DEFAULT_VISIBLE) -> None:
    """
    Wrap HeatMap in a FeatureGroup with a 'name' and 'show' flag so it appears in LayerControl.
    """
    pts = df[[LAT_COL, LON_COL]].dropna().values.tolist()
    if len(pts) >= HEATMAP_MIN_POINTS:
        fg = FeatureGroup(name=name, show=show)
        HeatMap(pts, radius=16, blur=20, max_zoom=9).add_to(fg)
        m.add_child(fg)

def build_map(df: pd.DataFrame, out_html: Path) -> None:
    center = [df[LAT_COL].median(), df[LON_COL].median()]
    # Use a clean non-Google basemap
    m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

    # All points (visible by default)
    add_points_layer(m, df, "All points", cluster=True, show=True)

    # Heatmap (toggable via LayerControl; default visibility set by HEATMAP_DEFAULT_VISIBLE)
    add_heatmap_layer(m, df, "Heatmap (All)", show=HEATMAP_DEFAULT_VISIBLE)

    # Category layers (top N) — each is togglable
    if CATEGORY_COL in df.columns:
        top_cats = (
            df[CATEGORY_COL]
            .value_counts(dropna=False)
            .head(5)
            .index.tolist()
        )
        for cat in top_cats:
            sub = df[df[CATEGORY_COL] == cat]
            add_points_layer(m, sub, f"Category: {cat}", cluster=True, show=False)

    # Time Period layers (top N) - each is togglable
    if TIME_PERIOD_COL in df.columns:
        top_times = (
            df[TIME_PERIOD_COL]
            .value_counts(dropna=False)
            .head(5)
            .index.tolist()
        )
        for time in top_times:
            sub = df[df[TIME_PERIOD_COL] == time]
            add_points_layer(m, sub, f"Time Period: {time}", cluster=True, show=False, color_col=TIME_PERIOD_COL)

    # Legend (category color swatches)
    if CATEGORY_COL in df.columns:
        cats = (
            df[CATEGORY_COL]
            .value_counts(dropna=False)
            .head(6)
            .index.tolist()
        )
        items = "".join(
            f'<div style="margin:4px 0;"><span style="display:inline-block;width:12px;height:12px;background:{color_for_category(c)};border:1px solid #333;margin-right:6px;"></span>{c}</div>'
            for c in cats
        )
        legend = f"""
        <div style="position: fixed; bottom: 24px; left: 24px; z-index: 9999;
            background: white; border: 1px solid #999; padding: 10px 12px; font-size: 13px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.15);">
            <b>{CATEGORY_COL} (top)</b>
            {items}
        </div>
        """
        m.get_root().html.add_child(folium.Element(legend))

    # Legend (Time Period color swatches)
    if TIME_PERIOD_COL in df.columns:
        times = (
            df[TIME_PERIOD_COL]
            .value_counts(dropna=False)
            .head(6)
            .index.tolist()
        )
        items = "".join(
            f'<div style="margin:4px 0;"><span style="display:inline-block;width:12px;height:12px;background:{color_for_category(c)};border:1px solid #333;margin-right:6px;"></span>{c}</div>'
            for c in times
        )
        legend = f"""
        <div style="position: fixed; bottom: 24px; left: 180px; z-index: 9999;
            background: white; border: 1px solid #999; padding: 10px 12px; font-size: 13px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.15);">
            <b>{TIME_PERIOD_COL} (top)</b>
            {items}
        </div>
        """
        m.get_root().html.add_child(folium.Element(legend))


    # Layer control (appears after we add all layers)
    folium.LayerControl(collapsed=False).add_to(m)

    # Fit bounds
    bounds = df[[LAT_COL, LON_COL]].dropna().values.tolist()
    if bounds:
        m.fit_bounds(bounds)

    m.save(str(out_html))

if __name__ == "__main__":
    df = load_df(CSV_PATH)
    # Example filters (uncomment/edit as needed):
    # df = filter_df(df, start="2025-01-01", end="2025-12-31",
    #                categories=["Accident","Traffic"], time_periods=["Morning"], ambulance=1)
    build_map(df, OUT_HTML)
    print(f"Saved: {OUT_HTML.resolve()}")

/tmp/ipython-input-849020714.py:37: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)
/tmp/ipython-input-849020714.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["event_dt"] = pd.to_datetime(dt_str, errors="coerce", infer_datetime_format=True)


Saved: /content/car_map_dashboard2.html
